In [1]:
import pandas as pd
ratings_movies = pd.read_csv('data/ratings_movies.csv')
display(ratings_movies)

,Unnamed: 0,userId,movieId,rating,date,title,genres
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...,...
100831,100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller
100832,100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror
100834,100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi


In [20]:
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [22]:
ratings_movies['year_release'] = ratings_movies['title'].apply(get_year_release)
ratings_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Unnamed: 0    100836 non-null  int64         
 1   userId        100836 non-null  int64         
 2   movieId       100836 non-null  int64         
 3   rating        100836 non-null  float64       
 4   date          100836 non-null  datetime64[ns]
 5   title         100836 non-null  object        
 6   genres        100836 non-null  object        
 7   Year          100836 non-null  int64         
 8   year_release  100818 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(2)
memory usage: 6.9+ MB


In [25]:
mask1 = ratings_movies['year_release'] == 1999
ratings_movies[mask1].groupby('title')['rating'].mean().sort_values()

title
Bloodsport: The Dark Kumite (1999)            0.5
Simon Sez (1999)                              1.0
Chill Factor (1999)                           1.0
Source, The (1999)                            1.0
Trippin' (1999)                               1.0
                                             ... 
Trailer Park Boys (1999)                      5.0
Larry David: Curb Your Enthusiasm (1999)      5.0
Sun Alley (Sonnenallee) (1999)                5.0
George Carlin: You Are All Diseased (1999)    5.0
Five Senses, The (1999)                       5.0
Name: rating, Length: 261, dtype: float64

In [40]:
mask2 = ratings_movies['year_release'] == 2010
ratings_movies[mask2].groupby('genres')['rating'].mean().sort_values()


genres
Action|Sci-Fi                        1.000000
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Adventure|Comedy|Fantasy             1.833333
                                       ...   
Crime                                4.750000
Comedy|Musical                       5.000000
Animation|Drama|Fantasy|Mystery      5.000000
Adventure|Children|Comedy|Mystery    5.000000
Animation|Children|Mystery           5.000000
Name: rating, Length: 119, dtype: float64

In [42]:
ratings_movies.groupby('userId')['genres'].nunique().sort_values(ascending=False)

userId
599    524
414    482
448    403
380    399
474    395
      ... 
578     15
12      15
85      13
214     13
245     13
Name: genres, Length: 610, dtype: int64

In [56]:
ratings_movies.groupby('userId')['rating'].agg(['count', 'mean']).sort_values(by=['count','mean'], ascending=[True,False])


,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


In [63]:
mask3 = ratings_movies['year_release'] == 2018
grouped = ratings_movies[mask3].groupby('genres')['rating'].agg(
    ['mean', 'count']
)
grouped[grouped['count']>10].sort_values(
    by='mean',
    ascending=False
)

,mean,count
genres,,
Action|Adventure|Sci-Fi,3.928571,14
Action|Comedy|Sci-Fi,3.875000,12


In [80]:
ratings_movies['date'] = pd.to_datetime(ratings_movies['date'])
ratings_movies['year_rating'] = ratings_movies['date'].dt.year
mask4 = ratings_movies['year_rating'] == 2018
mask5 = ratings_movies['genres'] == 'Animation|Children|Mystery'
ratings_movies[mask4 & mask5].pivot_table(
    values='rating',
    index='genres',
    columns='year_rating',
    fill_value=0
)

year_rating,2018
genres,
Animation|Children|Mystery,5


In [83]:
orders = pd.read_csv('data/orders.csv')
products = pd.read_csv('data/products.csv')
display(orders.info())

ParserError: Error tokenizing data. C error: Expected 1 fields in line 7, saw 2
